# Importing the European Roads dataset

In this notebook we're going to import the European Roads dataset into Neo4j. Let's first import the Neo4j Python driver:

In [24]:
from neo4j import GraphDatabase
from neo4j.exceptions import ClientError

Once we've done that we can initialise the Neo4j driver.

In [ ]:
driver = GraphDatabase.driver("bolt://localhost", auth=("neo4j", "neo"))

## Importing the dataset

And now we can import the dataset:

In [39]:
with driver.session(database="foo") as session:
    try: 
        session.run("""
        CREATE CONSTRAINT ON (p:Place) ASSERT p.name IS UNIQUE;
        """)
    except ClientError as ex:
        print(ex)
        
    result = session.run("""
    USING PERIODIC COMMIT 1000
    LOAD CSV WITH HEADERS FROM "https://github.com/neo4j-examples/graph-embeddings/raw/main/data/roads.csv"
    AS row

    MERGE (origin:Place {name: row.origin_reference_place})
    SET origin.countryCode = row.origin_country_code

    MERGE (destination:Place {name: row.destination_reference_place})
    SET destination.countryCode = row.destination_country_code

    MERGE (origin)-[eroad:EROAD {number: row.road_number}]->(destination)
    SET eroad.distance = toInteger(row.distance), eroad.watercrossing = row.watercrossing;
    """)
    display(result.consume().counters)

An equivalent constraint already exists, 'Constraint( UNIQUE, :Place(name) )'.


{'properties_set': 5000}